# Data splits

This notebook focuses on building the data splits to train the model, namely: train, validation and test.

*Constraint: every splits must contain unique subjects, namely different MR session for a subject x, needs to be contained in only ONE split*.

In [1]:
import pandas as pd
import os
import sys

# Append the path to access custom modules
if '../' not in sys.path:
    sys.path.append('../')

import src.utils.data_splitter as ds
import src.utils.dataset_helper as db
import importlib

In [2]:
importlib.reload(ds);
importlib.reload(db);

In [3]:
df = pd.read_csv('../data/csv/final_dataset.csv')
df = df.drop(columns='Unnamed: 0')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2475 entries, 0 to 2474
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Subject                       2475 non-null   object 
 1   MR_session                    2475 non-null   object 
 2   TOTAL_HIPPOCAMPUS_VOLUME      2475 non-null   float64
 3   Left-Hippocampus_volume       2475 non-null   float64
 4   lh_parahippocampal_thickness  2475 non-null   float64
 5   lh_parahippocampal_volume     2475 non-null   int64  
 6   rh_parahippocampal_volume     2475 non-null   int64  
 7   rh_parahippocampal_thickness  2475 non-null   float64
 8   Right-Hippocampus_volume      2475 non-null   float64
 9   label                         2475 non-null   object 
dtypes: float64(5), int64(2), object(3)
memory usage: 193.5+ KB


In [5]:
helper = db.DatasetHelper

## Undersample majority class

- Class distribution before resample

In [6]:
df['label'].value_counts()

label
Non-Demented    1981
MCI              381
Demented         113
Name: count, dtype: int64

- Get the undersampled dataset that will be the basis dataset from now on

In [ ]:
df = df.loc[helper.undersample_majority_class(df['label'], 700)]

- Class distribution after resample

In [8]:
df['label'].value_counts()

label
Non-Demented    700
MCI             381
Demented        113
Name: count, dtype: int64

- Create the csv file

In [9]:
df.to_csv('../data/csv/final_dataset_undersampled.csv', index=False)

- At this point we can create the new folder with the freesurfers whose MR label is in the resampled df

In [10]:
helper.move_folders(
    subject_experiments=df['MR_session'].to_list(), 
    base_dir=os.path.join('..', 'data', 'freesurfers'),
    dst_dir=os.path.join('..', 'data', 'freesurfers_undersampled')
)

## Define the test set

In [11]:
df = pd.read_csv('../data/csv/final_dataset_undersampled.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Subject                       1194 non-null   object 
 1   MR_session                    1194 non-null   object 
 2   TOTAL_HIPPOCAMPUS_VOLUME      1194 non-null   float64
 3   Left-Hippocampus_volume       1194 non-null   float64
 4   lh_parahippocampal_thickness  1194 non-null   float64
 5   lh_parahippocampal_volume     1194 non-null   int64  
 6   rh_parahippocampal_volume     1194 non-null   int64  
 7   rh_parahippocampal_thickness  1194 non-null   float64
 8   Right-Hippocampus_volume      1194 non-null   float64
 9   label                         1194 non-null   object 
dtypes: float64(5), int64(2), object(3)
memory usage: 93.4+ KB


- Instantiate the data splitter

In [13]:
splitter = ds.DataSplitterTrainTest(df['MR_session'], df['label'])

- Retrieve the the train and test instances

In [14]:
train_idx, test_idx = splitter.get_train_test_set_idx()

- Check train label distribution

In [16]:
df.loc[train_idx]['label'].value_counts()

label
Non-Demented    577
MCI             314
Demented         80
Name: count, dtype: int64

- Check test label distribution

In [17]:
df.loc[test_idx]['label'].value_counts()

label
Non-Demented    123
MCI              67
Demented         33
Name: count, dtype: int64

- Create a csv file related to test instances

In [18]:
df.loc[test_idx].to_csv('../data/csv/test.csv')

- Test volumetric images can now be moved in the dedicated folder

In [19]:
helper.move_folders(
    subject_experiments=df.loc[test_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'test_freesurfers')
)

## Define validation set

We can procede like before to define the validation set, to evaluate the model during training

In [20]:
# Work on a copy of the training set only
X_train = df.loc[train_idx].copy(deep=True) 

- Split the data

In [21]:
splitter = ds.DataSplitterTrainTest(X_train['MR_session'], X_train['label'])
train_idx, val_idx = splitter.get_train_test_set_idx()

In [22]:
X_train.loc[train_idx, 'label'].value_counts()

label
Non-Demented    487
MCI             268
Demented         61
Name: count, dtype: int64

In [23]:
X_train.loc[val_idx, 'label'].value_counts()

label
Non-Demented    90
MCI             46
Demented        19
Name: count, dtype: int64

- Move train set images to a separate folder

In [24]:
helper.move_folders(
    subject_experiments=X_train.loc[train_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'train_freesurfers')
)

- Move the validation set images to a separate folder

In [25]:
helper.move_folders(
    subject_experiments=X_train.loc[val_idx, 'MR_session'],
    base_dir=os.path.join('..', 'data', 'freesurfers_undersampled'),
    dst_dir=os.path.join('..', 'data', 'validation_freesurfers')
)

- Create clean (with no augmentated instances) csv file related to train and validation set

In [26]:
X_train.loc[train_idx].to_csv('../data/csv/train_clean.csv')
X_train.loc[val_idx].to_csv('../data/csv/validation_clean.csv')

## Add augmentation labels to train set

- This section will add to the train dataset (train + validation) the augmented instances to improve class balance
- Let's start by augmenting MCIs

In [27]:
mci_aug_train = helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'MCI'].sample(n=134, random_state=42),
    id_col_name='MR_session',
    suffix_list=['rot45']
)

mci_aug_val = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'MCI'].sample(n=23, random_state=42),
    id_col_name='MR_session',
    suffix_list=['rot45']
)

- Augment demented label

In [28]:
dem_aug_train = helper.augment_dataframe(
    df=X_train.loc[train_idx][X_train.loc[train_idx, 'label'] == 'Demented'],
    id_col_name='MR_session',
    suffix_list=['flip', 'rot30', 'rot90', 'rot150']
)

- In order to respect the proportion of classes, select a sample of 10 examples per agumentation, for the validation set
- Change the random state to avoid having the same 10 patients for each transformation

In [29]:
dem_aug_val_flip = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=42),
    id_col_name='MR_session',
    suffix_list=['flip']
)

dem_aug_val_rot30 = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=43),
    id_col_name='MR_session',
    suffix_list=['rot30']
)

dem_aug_val_rot90 = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=44),
    id_col_name='MR_session',
    suffix_list=['rot90']
)

dem_aug_val_rot150 = helper.augment_dataframe(
    df=X_train.loc[val_idx][X_train.loc[val_idx, 'label'] == 'Demented'].sample(n=10, random_state=45),
    id_col_name='MR_session',
    suffix_list=['rot150']
)

In [30]:
dem_aug_val = dem_aug_val_flip + dem_aug_val_rot30 + dem_aug_val_rot90 + dem_aug_val_rot150

- Get the final training dataframe

In [31]:
X_train_final = pd.concat(objs=[X_train.loc[train_idx]] + mci_aug_train + dem_aug_train, ignore_index=True)
X_val_final = pd.concat(objs=[X_train.loc[val_idx]] + mci_aug_val + dem_aug_val, ignore_index=True)

- This is the final distribution of the labels on the training set

In [32]:
X_train_final['label'].value_counts()

label
Non-Demented    487
MCI             402
Demented        305
Name: count, dtype: int64

- This is the final distribution of labels on the validation set

In [33]:
X_val_final['label'].value_counts()

label
Non-Demented    90
MCI             69
Demented        59
Name: count, dtype: int64

- Build a new column to indicate the augmentation operation to perform for a specific instance

In [34]:
X_train_final['augmentation'] = helper.extract_augmentation_column(X_train_final['MR_session'])
X_val_final['augmentation'] = helper.extract_augmentation_column(X_val_final['MR_session'])

In [37]:
X_train_final['MR_session_original'] = X_train_final['MR_session_original'].fillna(value='normal')
X_val_final['MR_session_original'] = X_val_final['MR_session_original'].fillna(value='normal')

- Save the augmented dataset to csv

In [38]:
X_train_final.to_csv('../data/csv/train.csv')
X_val_final.to_csv('../data/csv/validation.csv')

- The data augmentation step is performed in the next notebook, at the end of which the validation folder will be created.